# Hot Temperature Data Cleaning and Attribute Addition

This notebook will utilize prepared data to compute the "Feels Like" temperature.

This metric in weather prediction will add a weight to the high levels of humidity that can make a person lower the temperature in their thermostat.

My goal for this notebook is to add a new attribute to weather data and use the heat index equation to make a new temperature.

In [13]:
import pandas as pd
import numpy as np

# Load the data
miamiTotalData = pd.read_csv('WeatherData/HotnHumid/miamiSolar/miamiWeatherSolarTotal.csv')

miamiTotalData = miamiTotalData.drop(['Unnamed: 0'], axis=1)
miamiTotalData.head()

,Year,Month,Day,MaxTemp (F),MinTemp (F),DHI (w/m2),DNI (w/m2),Wind Speed (m/s),Relative Humidity (%),Pressure (mbar)
0,2015,1,1,84.0,66.0,61,159,2.1,85.12,1018
1,2015,1,2,84.0,69.0,54,215,4.6,85.98,1020
2,2015,1,3,83.0,74.0,79,122,6.7,84.27,1020
3,2015,1,4,85.0,71.0,49,240,5.1,84.11,1020
4,2015,1,5,85.0,67.0,32,309,3.2,86.44,1020


When dealing with temperatures in locations where there is a consistent high percentage of humidity, it is advised to use
the **heat index** equation. The **heat index** equation combines air temperature and relative humidity, in shaded areas, to posit a human-perceived equivalent temperature, as how hot it would feel if the humidity were some other value in the shade. 

The version of the [heat index equation](https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml) I will use is published from the National Oceanic and Atmospheric Administration (NOAA).

The NOAA supplies two versions of the **heat index equation** that vary between thresholds for both temperatures and relative humidities.

It is recommended that the full analysis of the **heat index equation** starts with Steadman's regression and then tested, to be recomputed with
Rothfusz's regression

In [14]:
from math import sqrt, fabs

def rothfusz_heat_index_regression(temperature, relative_humidity):
    """[Computes the 'feels like' temperature based on Rothfusz's regression]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature
    """
    T, RH = temperature, relative_humidity

    added_adjustment = ((RH - 85)/10) * ((87 - T)/5)
    subtracted_adjustment = ((13 - RH)/4) * sqrt(17 - fabs(T - 95) / 17)

    heat_index = (-42.379) + (2.04901523)*(T) + (10.14333127)*(RH) - (0.22475541)*(T)*(RH)
    heat_index = heat_index - (0.00683783)*(T)*(T) - (0.05481717)*(RH)*(RH) + (0.00122874)*(T)*(T)*(RH)
    heat_index = heat_index + (0.00085282)*(T)*(RH)*(RH) - (0.00000199)*(T)*(T)*(RH)*(RH)

    if ((RH < 13) and (80 < T < 112)):
        heat_index -= subtracted_adjustment
    elif ((RH > 85) and (80 < T < 87)):
        heat_index += added_adjustment

    return round(heat_index, 2)

In [15]:
def steadman_heat_index_regression(temperature, relative_humidity):
    """[Computes the 'feels like' temperature based on Steadman's regression]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature
    """
    T, RH = temperature, relative_humidity
    heat_index = (0.5) * (T + (61.0) + ((T - 68) * 1.2) + (RH * (0.094)))
    
    return round(heat_index, 2)

In practice, the Steadman's regression is computed first and the result averaged with the temperature.

**If** this heat index value is 80°F or higher, the full regression equation along with any adjustment as described above is applied.

In [50]:
def heat_index(temperature, relative_humidity):
    """[Computes the 'feels like' temperature for usage]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature based on NOAA's recommendation
    """
    T, RH = temperature, relative_humidity
    
    temp = steadman_heat_index_regression(temperature=T, relative_humidity=RH)
    steadman = (0.5) * (temp + T)

    rothfusz = rothfusz_heat_index_regression(temperature=T, relative_humidity=RH)

    if (steadman >= 80):
        return rothfusz
    
    return steadman

However, I noticed that some of our data is incomplete, as some values are missing.

We will apply a fill function that will take the average of the temperatures and fill the empty cells.

In [17]:
miamiTotalData.isnull().sum()

Year                      0
Month                     0
Day                       0
MaxTemp (F)              17
MinTemp (F)              16
DHI (w/m2)                0
DNI (w/m2)                0
Wind Speed (m/s)          0
Relative Humidity (%)     0
Pressure (mbar)           0
dtype: int64

Here, we are able to see that there are missing values in the temperature columns.

Since we have a lot of data, it'd be best if we just fill the missing values with the mean value for each of these columns.

In [18]:
miamiTotalData['MaxTemp (F)'].fillna(round(miamiTotalData['MaxTemp (F)'].mean(), 2), inplace=True)

In [19]:
miamiTotalData.isnull().sum()

Year                      0
Month                     0
Day                       0
MaxTemp (F)               0
MinTemp (F)              16
DHI (w/m2)                0
DNI (w/m2)                0
Wind Speed (m/s)          0
Relative Humidity (%)     0
Pressure (mbar)           0
dtype: int64

In [43]:
# For the minimum temperature there are some null values that we have to fill fortunately,
# there are not a lot, so we can just substitute those for the attribute's average
miamiTotalData['MinTemp (F)'].fillna(round(miamiTotalData['MinTemp (F)'].mean(), 2), inplace=True)

# We should be able to see that all the values from 'MinTemp (F)' are now filled
miamiTotalData.isnull().sum()

Year                     0
Month                    0
Day                      0
MaxTemp (F)              0
MinTemp (F)              0
Heat Index (F)           0
DHI (w/m2)               0
DNI (w/m2)               0
Wind Speed (m/s)         0
Relative Humidity (%)    0
Pressure (mbar)          0
dtype: int64

In [21]:
# Creating a new attribute in the dataframe, this new attribute will be the "feels like" temperature
miamiTotalData.insert(loc=5, column='Heat Index (F)', value=0, allow_duplicates=False)

Now we have added an attribute that will contain the **heat index** or **feels like** temperature. Our function to create the number will be applied to the whole column.

In [42]:
miamiTotalData['Heat Index (F)'] = miamiTotalData.apply(lambda row: heat_index(row['MaxTemp (F)'], row['Relative Humidity (%)']), axis=1)

miamiTotalData.tail()

,Year,Month,Day,MaxTemp (F),MinTemp (F),Heat Index (F),DHI (w/m2),DNI (w/m2),Wind Speed (m/s),Relative Humidity (%),Pressure (mbar)
1820,2019,12,27,82.00,72.0,91.28,44,247,5.2,88.14,1017
1821,2019,12,28,81.00,72.0,88.93,53,138,5.7,89.57,1017
1822,2019,12,29,80.00,73.0,85.46,62,119,5.5,88.79,1016
1823,2019,12,30,79.00,67.0,79.92,66,25,3.3,90.15,1014
1824,2019,12,31,85.82,59.0,100.81,44,170,3.5,83.05,1017


In [56]:
def user_heating(feels_like_temperature):
    min_heat = 60
    avg_heat = 63
    max_heat = 70



62.75
None
